In [197]:
# Импортируем необходимые библиотеки
import pandas as pd

# Загрузим файл xlsx и создадим датафрейм
df = pd.read_excel('/mnt/d/Doc_to_day/20240419/EntitiesEndpoints_fresh.xlsx')

# Проверим, что данные успешно загружены, например, выведем первые несколько строк
df.head(2)

,id,inn,address,entityId,type,addressId,regionCode,markingBranchId,markingStatus,warehouse,...,status,updateIncrement,data,source,createdAt,updatedAt,deletedAt,clientId,title,whExternalId
0,1014798,2126003123,"Чувашская Республика - чувашия, Мариинский Пос...",79601.0,NaN,NaN,NaN,NaN,NaN,False,...,clarification,1268546,{},pharmacy-sales,2024-04-14 20:22:58.158819 +00:00,2024-04-14 20:22:58.158819 +00:00,NaN,5a2d6114-b82a-4ac2-83ae-12ccc304f6fb,NaN,NaN
1,1014799,1434038369,"678995, Саха /Якутия/ Респ, Нерюнгри г, Серебр...",48020.0,NaN,NaN,NaN,NaN,NaN,False,...,clarification,1268547,{},pharmacy-sales,2024-04-14 20:22:58.158819 +00:00,2024-04-14 20:22:58.158819 +00:00,NaN,5a2d6114-b82a-4ac2-83ae-12ccc304f6fb,NaN,NaN


In [38]:
import re

def clean_address(address):
    if not address:
        return ''

    pharmacies = [
        'медикс', 'оптицена', 'аптека отличных цен', 'борте', 'социальная', 'твояаптека([,.\\s]+рф)?', 'семейная', 'гвоздика',
        'родиола', 'микма', 'анис', 'экона', 'черемуха', 'аптека фи', 'здоровье', 'детство', 'фармасервис', 'имбирь',
        'прогресс-фармаимпекс', 'цма', 'лекорт', 'медус', 'биопта', 'бирюза(-фарм)?', 'бионта', 'аптис', 'поволжье', 'моя аптека',
        'фонд', 'фарм плюс', 'бережная аптека', 'протей', 'тархун', 'диона', 'тефия', 'калипсо', 'биона', 'левзея',
        'орловские аптеки', 'ивика', 'душица', 'печора', 'енисей', 'двина', 'городская аптека', 'вереск',
        'сальма', 'вербена', 'фармаимпекс-юг', 'фармаимпекс-вятка', 'медда', 'пион', 'отлицена', 'таймыр', 'светлояр',
        'байкал', 'ладога', 'альтаир', 'фаркос', 'товары для здоровья', 'земляника', 'алтей', 'родиола', 'вистол',
        'брусника', 'арония', 'адонис',
    ]
    
    pharmacy1_regex = re.compile(r'аптека\s+({})\s+№[0-9]+'.format('|'.join(pharmacies)), re.IGNORECASE)
    pharmacy2_regex = re.compile(r'({})\s+№[0-9]+'.format('|'.join(pharmacies)), re.IGNORECASE)
    pharmacy3_regex = re.compile(r'({})[.,\s]*$'.format('|'.join(pharmacies)), re.IGNORECASE)

    cleaned_address = address.strip()
    cleaned_address = pharmacy1_regex.sub('', cleaned_address)
    cleaned_address = pharmacy2_regex.sub('', cleaned_address)
    cleaned_address = pharmacy3_regex.sub('', cleaned_address)
    cleaned_address = re.sub(r'^\d{6}[.,\s]+', '', cleaned_address)
    cleaned_address = re.sub(r'^Аптека[.,\s]+', '', cleaned_address)
    cleaned_address = re.sub(r'(^|[.,\s]+)[^., ]+\s+((м. *)?р-о?н|район)[.,\s]+?', ',', cleaned_address)
    cleaned_address = re.sub(r'(^|[.,\s]+)((м. *)?р-о?н|район)\s+[^., ]+[.,\s]+?', ',', cleaned_address)
    cleaned_address = re.sub(r'[.,\s]*\([а-яё]+ район\)[.,\s]*', ', ', cleaned_address, flags=re.IGNORECASE)
    cleaned_address = re.sub(r'[.,\s]*\(к.12 или к.13\)\s*', ', к.13', cleaned_address)
    cleaned_address = re.sub(r'[.,\s]+часть.*$', '', cleaned_address, flags=re.IGNORECASE)
    cleaned_address = re.sub(r'[.,\s]+неж(илое)?[^а-яё]\.?.*$', '', cleaned_address, flags=re.IGNORECASE)
    cleaned_address = re.sub(r'[.,\s]+расположенное.*$', '', cleaned_address, flags=re.IGNORECASE)
    cleaned_address = re.sub(r'[.,\s]+на террит.*$', '', cleaned_address, flags=re.IGNORECASE)
    cleaned_address = re.sub(r'[.,\s]+функциональное.*$', '', cleaned_address, flags=re.IGNORECASE)
    cleaned_address = re.sub(r'[.,\s]+(\(?аптечный\s*(п|к|$)|аптек[аи]).*$', '', cleaned_address, flags=re.IGNORECASE)
    cleaned_address = re.sub(r'[.,\s]+\(?не задано\)?\s*', '', cleaned_address, flags=re.IGNORECASE)
    cleaned_address = re.sub(r'[.,\sг]+\(?не использовать\)?\s*', '', cleaned_address, flags=re.IGNORECASE)
    cleaned_address = re.sub(r'[.,\s]+(городской округ|г\.? *о\.?)\s+[^.,\s]+', '', cleaned_address, flags=re.IGNORECASE)
    cleaned_address = re.sub(r'москва[. ]+обл[., ]+', 'Московская обл., ', cleaned_address, flags=re.IGNORECASE)
    cleaned_address = re.sub(r'тверь[. ]+обл[., ]+', 'Тверская обл., ', cleaned_address, flags=re.IGNORECASE)
    cleaned_address = re.sub(r'санкт-петербург[. ]+обл[., ]+', 'Ленинградская обл., ', cleaned_address, flags=re.IGNORECASE)
    cleaned_address = re.sub(r'рост[. ]+обл[., ]+', 'Ростовская обл., ', cleaned_address, flags=re.IGNORECASE)
    cleaned_address = re.sub(r'белгород[. ]+обл[., ]+', 'Белгородская обл., ', cleaned_address, flags=re.IGNORECASE)
    cleaned_address = re.sub(r'владимир[. ]+обл[., ]+', 'Владимирская обл., ', cleaned_address, flags=re.IGNORECASE)
    cleaned_address = re.sub(r'волгоград[. ]+обл[., ]+', 'Волгоградская обл., ', cleaned_address, flags=re.IGNORECASE)
    cleaned_address = re.sub(r'краснодар[. ]+обл[., ]+', 'Краснодарский край, ', cleaned_address, flags=re.IGNORECASE)
    cleaned_address = re.sub(r'(?:^|[ .,]+)спб([ .,]+|$)', 'г. Санкт-Петербург, ', cleaned_address, flags=re.IGNORECASE)
    cleaned_address = re.sub(r'(?:^|[ .,]+)мо([ .,]+|$)', '', cleaned_address, flags=re.IGNORECASE)
    cleaned_address = re.sub(r'новосибирск[. ]+обл[., ]+', 'Новосибирская обл., ', cleaned_address, flags=re.IGNORECASE)
    cleaned_address = re.sub(r'ростов[. ]+обл[., ]+', 'Ростовская обл., ', cleaned_address, flags=re.IGNORECASE)
    cleaned_address = re.sub(r'свердл[. ]+обл[., ]+', 'Свердловская обл., ', cleaned_address, flags=re.IGNORECASE)
    cleaned_address = re.sub(r'ставрополь[. ]+обл[., ]+', 'Ставропольский край, ', cleaned_address, flags=re.IGNORECASE)
    cleaned_address = re.sub(r'томск[. ]+обл[., ]+', 'Томская обл., ', cleaned_address, flags=re.IGNORECASE)
    cleaned_address = re.sub(r'тюмень?[. ]+обл[., ]+', 'Тюменская обл., ', cleaned_address, flags=re.IGNORECASE)
    cleaned_address = re.sub(r'челяб[. ]+обл[., ]+', 'Челябинская обл., ', cleaned_address, flags=re.IGNORECASE)
    cleaned_address = re.sub(r'челябинск[. ]+обл[., ]+', 'Челябинская обл., ', cleaned_address, flags=re.IGNORECASE)
    cleaned_address = re.sub(r'ярославль[. ]+обл[., ]+', 'Ярославская обл., ', cleaned_address, flags=re.IGNORECASE)
    cleaned_address = re.sub(r'(^|[.,\s()]+)мск[. ]+обл[., ]+', ' Московская обл., ', cleaned_address, flags=re.IGNORECASE)
    cleaned_address = re.sub(r'([0-9]+)-й', r'\1', cleaned_address)
    cleaned_address = re.sub(r'(лит(ера)?\.\s*[а-яё]+).*$', r'\1', cleaned_address, flags=re.IGNORECASE)
    cleaned_address = re.sub(r'[.,\s]+[0-9\s]+?-[нh].*$', '', cleaned_address)
    cleaned_address = re.sub(r'[.,\s(]+пом(?:ещени[ея])?\.?.*(ул.*)', r', \1', cleaned_address, flags=re.IGNORECASE)
    cleaned_address = re.sub(r'[.,\s(]+помещ\.?.*(ул.*)', r', \1', cleaned_address, flags=re.IGNORECASE)
    cleaned_address = re.sub(r'[.,\s(]+пом(ещени[ея])?\.?.*$', '', cleaned_address, flags=re.IGNORECASE)
    cleaned_address = re.sub(r'[.,\s(]+помещ\.?.*$', '', cleaned_address, flags=re.IGNORECASE)
    cleaned_address = re.sub(r'(^|[.,\s()]+)к\.?\s+стр\.?([().,\s]+|$)', r'\1 к. \2', cleaned_address)
    cleaned_address = re.sub(r'([.,\s(]+)стр(?:оен(?:ие)?)?\.?([.,\s0-9]+)', r'\1 стр. \2', cleaned_address)
    cleaned_address = re.sub(r'([0-9]+)\(?пом(ещени[ея])?\s*', r'\1, ', cleaned_address)
    cleaned_address = re.sub(r'([0-9]+)\(?помещ\s*', r'\1, ', cleaned_address)
    cleaned_address = re.sub(r'[.,\s]+Городское\s+поселение\s+[^.,]+', '', cleaned_address)
    cleaned_address = re.sub(r'[.,\s]+Внутригородская\s+территория\s+[^.,]+', '', cleaned_address)
    cleaned_address = re.sub(r'[.,\s]+[^.,]+\s+г\.о\.?', '', cleaned_address)
    cleaned_address = re.sub(r'[.,\s]+(кв|оф|офис|п|ком)\.?\s*[0-9ivxlcdm]+.*$', '', cleaned_address)
    cleaned_address = re.sub(r'[.,\s]+(поз|комнаты)\.?\s*[0-9-]+.*(ул|$)', '', cleaned_address)
    cleaned_address = re.sub(r'(?:^|[ .,]+)спб([ .,]+|$)', 'г. Санкт-Петербург, ', cleaned_address, flags=re.IGNORECASE)
    cleaned_address = re.sub(r'(?:^|[ .,]+)мо([ .,]+|$)', '', cleaned_address, flags=re.IGNORECASE)
    cleaned_address = re.sub(r'м\/о[^.,]+([ .,]+|$)', '', cleaned_address, flags=re.IGNORECASE)
    cleaned_address = re.sub(r'[ .,]+(?:окей|максидом|уч. ж\/д.+|бывш. лит.+|метро.+)[ .,]+?', '', cleaned_address, flags=re.IGNORECASE)
    cleaned_address = re.sub(r'\([^(]\)$', '', cleaned_address)
    cleaned_address = re.sub(r'\(.*?\)', '', cleaned_address)
    cleaned_address = re.sub(r'[.,\s]*"?торговый (центр|павильо([а-яё])?)"?[.,\s]*', '', cleaned_address, flags=re.IGNORECASE)
    cleaned_address = re.sub(r'[.,\s]*"?парикмахерская"?[.,\s]*', '', cleaned_address, flags=re.IGNORECASE)
    cleaned_address = re.sub(r'([0-9]+)-й', r'\1', cleaned_address)
    cleaned_address = re.sub(r'(^|[.,\s()]+)(?:с\.\s+п\.|с\/п|сп)([().,\s]+|$)', r'\1 \2', cleaned_address, flags=re.IGNORECASE)
    cleaned_address = re.sub(r'[.,\s(]+бокс.*$', '', cleaned_address, flags=re.IGNORECASE)
    cleaned_address = re.sub(r'([.,\s(]+)напротив дома(.*)', r'\1 д. \2', cleaned_address, flags=re.IGNORECASE)
    cleaned_address = re.sub(r'[.,\s(]+(тц[^а-яё]|жк[^а-яё]|тк[^а-яё]|у +платформы|магазин|мини-рынок).*$', '', cleaned_address, flags=re.IGNORECASE)
    cleaned_address = re.sub(r'(^|[.,\s()]+)(м. *)?р-он([().,\s]+|$)', r'\1 р-н \2', cleaned_address, flags=re.IGNORECASE)
    cleaned_address = re.sub(r'(^|[.,\s()]+)домовл.?([().,\s]+|$)', r'\1 \2', cleaned_address, flags=re.IGNORECASE)
    cleaned_address = re.sub(r'(^|[.,\s()]+)имени([().,\s]+|$)', r'\1 \2', cleaned_address, flags=re.IGNORECASE)
    cleaned_address = re.sub(r'[.,\s]+нет[.,\s]', ', ', cleaned_address, flags=re.IGNORECASE)
    cleaned_address = re.sub(r'^\/[0-9]+(_[^ ]+)?', ', ', cleaned_address)
    cleaned_address = re.sub(r'[.,\s()]+корп([\s.,]+)?$', '', cleaned_address, flags=re.IGNORECASE)
    cleaned_address = re.sub(r'[.,\s]*аптека[.,\s]*', ', ', cleaned_address, flags=re.IGNORECASE)
    cleaned_address = re.sub(r'(\bул\.\s*\w+)\s*(д\.|дом)\s*(\d+)', r'\1, \3', cleaned_address, flags=re.IGNORECASE)
    cleaned_address = re.sub(r'(д\.|дом)\s*(\d+)', r'\1 \2', cleaned_address, flags=re.IGNORECASE)
    cleaned_address = re.sub(r'\bг(ор)?\.?\s*', 'г. ', cleaned_address, flags=re.IGNORECASE)
    cleaned_address = re.sub(r'(д\.|дом|д)\s*(\d+)', r'д \2', cleaned_address, flags=re.IGNORECASE)
    cleaned_address = re.sub(r'\bпавильон\s|\bпавильон[.,]?\s?|\bпав\b|\bп-н\b|\bпавиль\b|\bпавиль[.]\s?', '', cleaned_address)
    cleaned_address = re.sub(r'\b\d+\s*этаж\b\s*,?', '', cleaned_address)
    
    #cleaned_address = re.sub(r'(?:^|[ .,]+)(г|гор|город|п|пос|поселок|с|село|дер|деревня|пгт)(?:[ .,]+|$)', '', cleaned_address, flags=re.IGNORECASE)


    if not re.search(r'(?:^|[.,\s()]+)яя([().,\s]+|$)', cleaned_address, flags=re.IGNORECASE):
        cleaned_address = re.sub(r'^[а-яё]{2}[., ]+', ', ', cleaned_address, flags=re.IGNORECASE)

    cleaned_address = re.sub(r'[a-z]+', '', cleaned_address)
    cleaned_address = re.sub(r'\s*\.\s*', '. ', cleaned_address)
    cleaned_address = re.sub(r'\s*,\s*', ', ', cleaned_address)
    cleaned_address = re.sub(r'[.,\s]+$', '', cleaned_address)
    cleaned_address = re.sub(r'^[.,\s]+', '', cleaned_address)
    cleaned_address = re.sub(r'\s+', ' ', cleaned_address).strip()

    return cleaned_address

In [199]:
# Напишем функцию для проверки адреса по заданным условиям
def check_broken_address(address):
    # Проверяем условие 1: не содержать цифр после обработки
    if not re.search(r'\d', address):
        return True
    # Проверяем условие 2: пусто или только символы
    if re.search(r'^\s*$', address):
        return True
    # Проверяем условие 3: нет букв, только цифры
    if re.search(r'^\d+$', address):
        return True
    return False



In [171]:
def split_joined_address(address):
    # Проверяем, содержит ли адрес хотя бы одну букву и одну цифру
    if re.search(r'[а-яёА-ЯЁ].*\d|\d.*[а-яёА-ЯЁ]', address):
        # Проверяем, является ли адрес слипшимся
        if len(re.findall(r'[А-ЯЁа-яё]', address)) > len(address) // 2:
            # Разделяем адрес только если он слипшийся
            split_address = re.split(r'(?<=[а-яё])(?=[0-9])|(?<=[0-9])(?=[а-яё])|(?<=[а-яё])(?=[А-ЯЁ])|(?<=[А-ЯЁ])(?=[а-яё])', address)
            return ' '.join(split_address)
    return address

In [200]:
# Пример использования:
sample_address = "Аптека Медикс №123, ул. Пушкина, д. 10, корп. 2, Москва кв 18"
cleaned_sample_address = clean_address(sample_address)
print(cleaned_sample_address)

ул. Пушкина, д 10, корп. 2, Москва


In [201]:
# Применение функции к каждой строке в столбце адресов
df.insert(df.columns.get_loc('address') + 1, 'address_clear', df['address'].apply(clean_address))

# Отображение результата
df.head(2)

,id,inn,address,address_clear,entityId,type,addressId,regionCode,markingBranchId,markingStatus,...,status,updateIncrement,data,source,createdAt,updatedAt,deletedAt,clientId,title,whExternalId
0,1014798,2126003123,"Чувашская Республика - чувашия, Мариинский Пос...","Чувашская Республика - чувашия, Мариинский Пос...",79601.0,NaN,NaN,NaN,NaN,NaN,...,clarification,1268546,{},pharmacy-sales,2024-04-14 20:22:58.158819 +00:00,2024-04-14 20:22:58.158819 +00:00,NaN,5a2d6114-b82a-4ac2-83ae-12ccc304f6fb,NaN,NaN
1,1014799,1434038369,"678995, Саха /Якутия/ Респ, Нерюнгри г, Серебр...","Саха /Якутия/ Респ, Нерюнгри г., Серебряный Бо...",48020.0,NaN,NaN,NaN,NaN,NaN,...,clarification,1268547,{},pharmacy-sales,2024-04-14 20:22:58.158819 +00:00,2024-04-14 20:22:58.158819 +00:00,NaN,5a2d6114-b82a-4ac2-83ae-12ccc304f6fb,NaN,NaN


In [202]:
# Находим индекс столбца 'address_clear'
address_clear_index = df.columns.get_loc('address_clear')

# Добавляем столбец 'broken_address' после 'address_clear'
df.insert(address_clear_index + 1, 'broken_address', df['address_clear'].apply(check_broken_address))

In [203]:
df.sample(2)

,id,inn,address,address_clear,broken_address,entityId,type,addressId,regionCode,markingBranchId,...,status,updateIncrement,data,source,createdAt,updatedAt,deletedAt,clientId,title,whExternalId
1991,1013252,6630006059,"Магнитогорск, К.Маркса 172 (Ашан)","Магнитогорск, К. Маркса 172",False,74461.0,NaN,NaN,NaN,NaN,...,clarification,1267000,{},pharmacy-sales,2024-04-14 20:22:58.158819 +00:00,2024-04-14 20:22:58.158819 +00:00,NaN,5a2d6114-b82a-4ac2-83ae-12ccc304f6fb,NaN,NaN
51728,602434,7721851829,"143070, Московская обл., Одинцовский р-н, г. К...","Московская обл, г. Кубинка, в/ч 28337, КДЦ",False,26746.0,NaN,NaN,NaN,NaN,...,clarification,811410,{},cislink-client-59301754,2024-03-05 19:26:34.015547 +00:00,2024-03-05 19:26:34.015547 +00:00,NaN,5a2d6114-b82a-4ac2-83ae-12ccc304f6fb,NaN,NaN


In [179]:
# Применяем функцию split_joined_address только к адресам, для которых флаг broken_address установлен в True
df.loc[df['broken_address'] == True, 'address_clear'] = df.loc[df['broken_address'] == True, 'address_clear'].apply(split_joined_address)


In [193]:
df.sample(10)

,id,inn,address,address_clear,broken_address,entityId,type,addressId,regionCode,markingBranchId,...,status,updateIncrement,data,source,createdAt,updatedAt,deletedAt,clientId,title,whExternalId
62139,615755,7839300921,№142 склад недовозов,№142 склад недовозов,False,6613.0,NaN,NaN,NaN,NaN,...,clarification,824731,{},cislink-distr-917656,2024-03-05 19:27:49.144976 +00:00,2024-03-05 19:27:49.144976 +00:00,NaN,5a2d6114-b82a-4ac2-83ae-12ccc304f6fb,NaN,NaN
47907,598260,663201357297,-,-,True,69535.0,NaN,NaN,NaN,NaN,...,clarification,807236,{},cislink-client-71598796,2024-03-05 19:26:34.015547 +00:00,2024-03-05 19:26:34.015547 +00:00,NaN,5a2d6114-b82a-4ac2-83ae-12ccc304f6fb,NaN,NaN
23398,1007785,1657198140,"420066, Республика Татарстан, Лаишевский муниц...","Республика Татарстан, Лаишевский, Столбищенско...",False,61161.0,NaN,NaN,NaN,00000000533050,...,clarification,1216762,{},cislink-mdlp-turnovers-00000000533050,2024-04-02 13:11:23.367665 +00:00,2024-04-02 13:11:23.367665 +00:00,NaN,5a2d6114-b82a-4ac2-83ae-12ccc304f6fb,NaN,NaN
49638,600183,9999999999,"453330, Башкортостан Респ, Кугарчинский р-н, М...","Башкортостан Респ, МраковоХудайбердина ул, д 51",False,5610.0,NaN,NaN,NaN,NaN,...,clarification,809159,{},cislink-client-35416108,2024-03-05 19:26:34.015547 +00:00,2024-03-05 19:26:34.015547 +00:00,NaN,5a2d6114-b82a-4ac2-83ae-12ccc304f6fb,NaN,NaN
61426,613055,6367052981,"ООО ""ДЕЛЬТА"" Интернет магазин","ООО ""ДЕЛЬТА"" Интернет",True,73119.0,NaN,NaN,NaN,NaN,...,clarification,822031,{},cislink-distr-883465,2024-03-05 19:27:49.144976 +00:00,2024-03-05 19:27:49.144976 +00:00,NaN,5a2d6114-b82a-4ac2-83ae-12ccc304f6fb,NaN,NaN
11661,1021222,6321468786,"445040, Самарская обл, Тольятти г, Ленинский п...","Самарская обл, ТольяттиЛенинский пр-кт/Туполев...",False,3466.0,NaN,NaN,NaN,NaN,...,clarification,1274972,{},pharmacy-sales,2024-04-14 21:14:32.576916 +00:00,2024-04-14 21:14:32.576916 +00:00,NaN,5a2d6114-b82a-4ac2-83ae-12ccc304f6fb,NaN,NaN
45271,595387,7707443151,"108809, МАРУШКИНСКОЕ С/П, Г МОСКВА, КВАРТАЛ № ...","МАРУШКИНСКОЕМОСКВА, КВАРТАЛ № 63, ДВЛд 1, СТР....",False,7047.0,NaN,NaN,NaN,NaN,...,clarification,804363,{},cislink-client-73363415,2024-03-05 19:26:34.015547 +00:00,2024-03-05 19:26:34.015547 +00:00,NaN,5a2d6114-b82a-4ac2-83ae-12ccc304f6fb,NaN,NaN
36550,585599,6453142117,-,-,True,70233.0,NaN,NaN,NaN,NaN,...,clarification,794575,{},cislink-client-70475952,2024-03-05 19:26:34.015547 +00:00,2024-03-05 19:26:34.015547 +00:00,NaN,5a2d6114-b82a-4ac2-83ae-12ccc304f6fb,NaN,NaN
34891,583699,9999999999,"603142, Нижегородская обл, г. Нижний Новгород,...","Нижегородская облНижний Новгород, шоссе Южное,...",False,5610.0,NaN,NaN,NaN,NaN,...,clarification,792675,{},cislink-client-35416437,2024-03-05 19:26:34.015547 +00:00,2024-03-05 19:26:34.015547 +00:00,NaN,5a2d6114-b82a-4ac2-83ae-12ccc304f6fb,NaN,NaN
50387,600983,9999999999,"461500, Оренбургская обл, Соль-Илецкий р-н, Со...","Оренбургская обл, Соль-ИлецкУральская ул, д 52",False,5610.0,NaN,NaN,NaN,NaN,...,clarification,809959,{},cislink-client-73497093,2024-03-05 19:26:34.015547 +00:00,2024-03-05 19:26:34.015547 +00:00,NaN,5a2d6114-b82a-4ac2-83ae-12ccc304f6fb,NaN,NaN


In [195]:
 Сохраняем датафрейм в файл Excel
df.to_excel('/mnt/d/Doc_to_day/20240419/EntitiesEndpoints_fresh_cleaned.xlsx', index=False)
#

Часть 2 - обработка уже полученного файла через Наташу


In [48]:
# Импортируем необходимые библиотеки
import pandas as pd

# Загрузим файл xlsx и создадим датафрейм
df = pd.read_excel('/mnt/d/Doc_to_day/20240419/EntitiesEndpoints_fresh_cleaned.xlsx')

# Проверим, что данные успешно загружены, например, выведем первые несколько строк
df.head(2)

,id,inn,address,address_clear,broken_address,new_fias,id_from,address_dadata,entityId,type,...,status,updateIncrement,data,source,createdAt,updatedAt,deletedAt,clientId,title,whExternalId
0,1014798,2126003123,"Чувашская Республика - чувашия, Мариинский Пос...","Чувашская Республика - чувашия, Мариинский Пос...",False,NaN,NaN,NaN,79601.0,NaN,...,clarification,1268546,{},pharmacy-sales,2024-04-14 20:22:58.158819 +00:00,2024-04-14 20:22:58.158819 +00:00,NaN,5a2d6114-b82a-4ac2-83ae-12ccc304f6fb,NaN,NaN
1,1014799,1434038369,"678995, Саха /Якутия/ Респ, Нерюнгри г, Серебр...","Саха /Якутия/ Респ, Нерюнгри г., Серебряный Бо...",False,NaN,NaN,Address not found,48020.0,NaN,...,clarification,1268547,{},pharmacy-sales,2024-04-14 20:22:58.158819 +00:00,2024-04-14 20:22:58.158819 +00:00,NaN,5a2d6114-b82a-4ac2-83ae-12ccc304f6fb,NaN,NaN


In [ ]:
df.info()

In [49]:
# Установим условие фильтрации только для столбца broken_address
condition = df['broken_address'] == False & df['new_fias'].isnull()

# Применим фильтр к данным и получим только строки, удовлетворяющие условию
filtered_df = df[condition]

# Получим количество отфильтрованных строк
count = filtered_df.shape[0]

# Выведем количество отфильтрованных строк
print("Количество строк, удовлетворяющих условиям: ", count)

Количество строк, удовлетворяющих условиям:  48363


In [24]:
# Теперь посчитаем количество значений в столбце address
address_count = filtered_df['address'].count()

# Выведем количество значений в столбце address
print("Количество значений в столбце address:", address_count)

Количество значений в столбце address: 48363


In [52]:
df.head(10)

,id,inn,address,aaddress_info_natasha,address_clear,broken_address,new_fias,id_from,address_dadata,entityId,...,status,updateIncrement,data,source,createdAt,updatedAt,deletedAt,clientId,title,whExternalId
0,1014798,2126003123,"Чувашская Республика - чувашия, Мариинский Пос...","Чувашская, республика; Николаева, улица; 54, дом","Чувашская Республика - чувашия, Мариинский Пос...",False,NaN,NaN,NaN,79601.0,...,clarification,1268546,{},pharmacy-sales,2024-04-14 20:22:58.158819 +00:00,2024-04-14 20:22:58.158819 +00:00,NaN,5a2d6114-b82a-4ac2-83ae-12ccc304f6fb,NaN,NaN
1,1014799,1434038369,"678995, Саха /Якутия/ Респ, Нерюнгри г, Серебр...","678995, индекс; № 304, дом","Саха /Якутия/ Респ, Нерюнгри г., Серебряный Бо...",False,NaN,NaN,Address not found,48020.0,...,clarification,1268547,{},pharmacy-sales,2024-04-14 20:22:58.158819 +00:00,2024-04-14 20:22:58.158819 +00:00,NaN,5a2d6114-b82a-4ac2-83ae-12ccc304f6fb,NaN,NaN
2,1011537,5906086550,"618909, Пермский край, Лысьвенский городской о...","618909, индекс; Пермский, край; Лысьва, город;...","Пермский край, Лысьвенский г. одской округ, г....",False,NaN,NaN,Address not found,5925.0,...,clarification,1265285,{},cislink-mdlp-rests-00000000453446,2024-04-14 08:15:42.480473 +00:00,2024-04-14 08:15:42.480473 +00:00,NaN,5a2d6114-b82a-4ac2-83ae-12ccc304f6fb,NaN,NaN
3,1014800,5902199785,"Екб, Верхняя Пышма, Успенский пр-кт., 20","Успенский, проспект","Екб, Верхняя Пышма, Успенский пр-кт., 20",False,NaN,NaN,Address not found,75366.0,...,clarification,1268548,{},pharmacy-sales,2024-04-14 20:22:58.158819 +00:00,2024-04-14 20:22:58.158819 +00:00,NaN,5a2d6114-b82a-4ac2-83ae-12ccc304f6fb,NaN,NaN
4,1011538,4217146394,"654054, Кемеровская область - Кузбасс, г. Ново...","654054, индекс; Кемеровская, область; Новокузн...","Кемеровская область - Кузбасс, г. Новокузнецк,...",False,NaN,NaN,Address not found,78312.0,...,clarification,1265286,{},cislink-mdlp-rests-00000000484603,2024-04-14 08:15:42.480473 +00:00,2024-04-14 08:15:42.480473 +00:00,NaN,5a2d6114-b82a-4ac2-83ae-12ccc304f6fb,NaN,NaN
5,1011539,4217178886,"654044, Кемеровская область - Кузбасс, Новокуз...","654044, индекс; Кемеровская, область; Новокузн...","Кемеровская область - Кузбасс, Новокузнецкий г...",False,NaN,NaN,Address not found,11098.0,...,clarification,1265287,{},cislink-mdlp-rests-00000000499544,2024-04-14 08:15:42.480473 +00:00,2024-04-14 08:15:42.480473 +00:00,NaN,5a2d6114-b82a-4ac2-83ae-12ccc304f6fb,NaN,NaN
6,1011540,6367650886,"446541, Самарская область, Сергиевский район, ...","446541, индекс; Самарская, область; Сергиевски...","Самарская область, с. Сергиевск, ул. Ленина, д...",False,NaN,NaN,Address not found,10078.0,...,clarification,1265288,{},cislink-mdlp-rests-00000000170841,2024-04-14 08:15:42.480473 +00:00,2024-04-14 08:15:42.480473 +00:00,NaN,5a2d6114-b82a-4ac2-83ae-12ccc304f6fb,NaN,NaN
7,1014801,1001054160,"г.Петрозаводск, Лесной проспект( ТЦ Лотос-Плаз...","Петрозаводск, город; Лесной, проспект","г. Петрозаводск, Лесной проспект",True,NaN,NaN,NaN,7018.0,...,clarification,1268549,{},pharmacy-sales,2024-04-14 20:22:58.158819 +00:00,2024-04-14 20:22:58.158819 +00:00,NaN,5a2d6114-b82a-4ac2-83ae-12ccc304f6fb,NaN,NaN
8,1011541,7204157965,"629830, Ямало-Ненецкий автономный округ, г. Гу...","629830, индекс; Ямало-Ненецкий, автономный окр...","Ямало-Ненецкий автономный округ, г. г. убкинск...",False,NaN,NaN,Address not found,80074.0,...,clarification,1265289,{},cislink-mdlp-rests-00000000141953,2024-04-14 08:15:42.480473 +00:00,2024-04-14 08:15:42.480473 +00:00,NaN,5a2d6114-b82a-4ac2-83ae-12ccc304f6fb,NaN,NaN
9,1011543,320301950920,"000000, Брянская область, Карачевский район, г...","Брянская, область; Карачевский, район; Карачев...","Брянская область, г. Карачев, ул. Первомайская...",False,NaN,NaN,Address not found,76724.0,...,clarification,1265291,{},cislink-mdlp-rests-00000000471265,2024-04-14 08:15:42.480473 +00:00,2024-04-14 08:15:42.480473 +00:00,NaN,5a2d6114-b82a-4ac2-83ae-12ccc304f6fb,NaN,NaN


In [50]:
from natasha import AddrExtractor, MorphVocab

# Создаем экземпляр MorphVocab
morph_vocab = MorphVocab()

# Создаем экземпляр AddrExtractor с передачей объекта MorphVocab
addr_extractor = AddrExtractor(morph_vocab)

# Функция для извлечения адреса из текста
def extract_address_info(text):
    matches = addr_extractor(text)
    if matches:
        address_info = []
        for match in matches:
            address_parts = []
            for part in match.fact.as_json.values():
                address_parts.append(str(part))
            address_info.append(", ".join(address_parts))
        return "; ".join(address_info)
    else:
        return None

# Применяем функцию к первым 10 строкам столбца address и создаем новый столбец aaddress_info_natasha
df.insert(df.columns.get_loc('address') + 1, 'aaddress_info_natasha', df['address'].apply(extract_address_info))

# Выводим первые 10 строк с новым столбцом
print(df[['address', 'aaddress_info_natasha']])


                                                 address  \
0      Чувашская Республика - чувашия, Мариинский Пос...   
1      678995, Саха /Якутия/ Респ, Нерюнгри г, Серебр...   
2      618909, Пермский край, Лысьвенский городской о...   
3               Екб, Верхняя Пышма, Успенский пр-кт., 20   
4      654054, Кемеровская область - Кузбасс, г. Ново...   
...                                                  ...   
68120  390007, Рязанская обл., г. Рязань, ул. Октябрь...   
68121                                                  -   
68122                                                  -   
68123  394075, Воронежская область, Воронеж г, Чудесн...   
68124  461041, Оренбургская область, Бузулук г, Фрунз...   

                                   aaddress_info_natasha  
0       Чувашская, республика; Николаева, улица; 54, дом  
1                             678995, индекс; № 304, дом  
2      618909, индекс; Пермский, край; Лысьва, город;...  
3                                    Успенс

In [ ]:
def swap_number_and_house(text):
    # Разбиваем адрес на части, используя разделитель ";"
    parts = text.split(";")
    if len(parts) > 1:
        # Разделяем последнюю часть адреса на слова
        last_part = parts[-1].split()
        # Проверяем, есть ли в последней части "дом" и номер
        if "дом" in last_part and last_part.index("дом") > 0:
            # Находим индекс "дом" и меняем местами предыдущее и последующее слово,
            # если это возможно
            idx = last_part.index("дом")
            if idx - 1 >= 0 and idx + 1 < len(last_part):
                last_part[idx-1], last_part[idx+1] = last_part[idx+1], last_part[idx-1]
                # Обновляем последнюю часть адреса
                parts[-1] = " ".join(last_part)
                # Обновляем адрес, объединяя части обратно в строку
                return "; ".join(parts)
    return text

In [44]:
# Импортируем необходимые библиотеки
import pandas as pd

# Загрузим файл xlsx и создадим датафрейм
df = pd.read_excel('/mnt/d/Doc_to_day/20240419/EntitiesEndpoints_fresh_cleaned_step_2.xlsx')

# Проверим, что данные успешно загружены, например, выведем первые несколько строк
df.head(2)

,id,inn,address,address_clear,broken_address,new_fias,id_from,address_dadata,entityId,type,...,status,updateIncrement,data,source,createdAt,updatedAt,deletedAt,clientId,title,whExternalId
0,1014798,2126003123,"Чувашская Республика - чувашия, Мариинский Посад г, Николаева ул, дом 54","Чувашская Республика - чувашия, Мариинский Посад г., Николаева ул, д 54",False,NaN,NaN,NaN,79601.0,NaN,...,clarification,1268546,{},pharmacy-sales,2024-04-14 20:22:58.158819 +00:00,2024-04-14 20:22:58.158819 +00:00,NaN,5a2d6114-b82a-4ac2-83ae-12ccc304f6fb,NaN,NaN
1,1014799,1434038369,"678995, Саха /Якутия/ Респ, Нерюнгри г, Серебряный Бор п, дом № 304","Саха /Якутия/ Респ, Нерюнгри г., Серебряный Бор п, дом № 304",False,NaN,NaN,Address not found,48020.0,NaN,...,clarification,1268547,{},pharmacy-sales,2024-04-14 20:22:58.158819 +00:00,2024-04-14 20:22:58.158819 +00:00,NaN,5a2d6114-b82a-4ac2-83ae-12ccc304f6fb,NaN,NaN


In [43]:
import re

def normalize_address(address, region_dict, city_dict, street_dict, building_dict, district_dict, village_dict):
    # Применение словарей к адресу
    for key, value in region_dict.items():
        address = address.replace(key, value)
    for key, value in city_dict.items():
        address = address.replace(key, value)
    for key, value in street_dict.items():
        address = re.sub(r'\b{}\b'.format(re.escape(key)), value, address)
    for key, value in building_dict.items():
        address = re.sub(r'\b{}\b'.format(re.escape(key)), value, address)
    for key, value in district_dict.items():
        address = re.sub(r'\b{}\b'.format(re.escape(key)), value, address)
    for key, value in village_dict.items():
        address = re.sub(r'\b{}\b'.format(re.escape(key)), value, address)

    return address

# Создание словарей для нормализации
region_dict = {
    "Московская обл": "Московская область",
    "Краснодарский край": "Краснодарский край",
    "область": "обл",
    "обл.": "обл",
    "обл ": "обл ",
    "обл,": "обл,",
    "обл. ": "обл ",
    "об-ть": "обл",
    "об.-ть": "обл",
    "об. -ть": "обл",
    "об-во": "обл",
    "об.-во": "обл",
    "об. -во": "обл",
    "област": "обл",
    "област.": "обл",
    "обл-сть": "обл",
    "област-ть": "обл",
    "област.-ть": "обл",
    "обл-та": "обл",
    "областа": "обл",
    "област-та": "обл",
    "обл,": "обл",
    "обл-цо": "обл",
    "области": "обл",
    "областное": "обл",
}

city_dict = {
    "город": "г",
    "г.": "г",
    "г ": "г ",
    "г,": "г,",
    "г. ": "г ",
    "г-д": "г",
    "г.-д": "г",
    "г. -д": "г",
    "грд": "г",
    "гр": "г",
    "г.р.": "г",
    "г. р.": "г",
    "г.-р.": "г",
    "гр.": "г",
    "г.-р": "г",
    "г.-р.": "г",
    "г": "г", 
    "год": "г",
    "годок": "г",
    "град": "г",
    "гр.": "г",
    "г. ": "г ",
    "г.р": "г",
}

street_dict = {
    "улица": "ул",
    "ул.": "ул",
    "ул ": "ул ",
    "ул,": "ул,",
    "ул. ": "ул ",
    "улиц": "ул",
    "улица ": "ул ",
    "ул-ца": "ул",
    "ул.-ца": "ул",
    "ул. -ца": "ул",
    "улец": "ул",
    "улес": "ул",
    "улек": "ул",
    "улица-": "ул",
    "улица -": "ул",
    "улетц": "ул",
    "улец-ца": "ул",
    "улес-ца": "ул",
    "улек-ца": "ул",
    "ул-ц": "ул",
}

district_dict = {
    "район": "р-н",
    "р-н": "р-н",
    "р-н.": "р-н",
    "р-н ": "р-н ",
    "р-н,": "р-н,",
    "р-н. ": "р-н ",
    "р-н -": "р-н ",
    "р-н-": "р-н ",
    "рн": "р-н",
    "р.н": "р-н",
    "р-но": "р-н",
    "рн.": "р-н",
    "р-на": "р-н",
    "р-ну": "р-н",
    "р-ном": "р-н",
    "р-нe": "р-н",
    "р-не": "р-н",
    "р-нa": "р-н",
    "р-нах": "р-н",
    "рай": "р-н",
    "районе": "р-н",
    "району": "р-н",
    "районом": "р-н",
    "района": "р-н",
    "районный": "р-н",
    "районная": "р-н",
    "районно": "р-н",
    "районные": "р-н",
    "районных": "р-н",
}

building_dict = {
    "строение": "стр",
    "стр.": "стр",
    "строен": "стр",
    "строен.": "стр",
    "строение ": "стр ",
    "строение,": "стр,",
    "строение. ": "стр ",
    "строен. ": "стр ",
    "строен,": "стр,",
    "строен ": "стр ",
    "строению": "стр",
    "стр-е": "стр",
    "строен.": "стр",
    "стр. -": "стр ",
    "стр-к": "стр",
    "строенка": "стр",
    "строен. ": "стр ",
    "строен,": "стр,",
    "строения": "стр",
    "строений": "стр",
    "строением": "стр",
    "строениям": "стр",
    "строению": "стр",
    "строением": "стр",
    "строениями": "стр",
    "строении": "стр",
}

village_dict = {
    "с": "село",
    "с.": "село",
    "с ": "село ",
    "с,": "село,",
    "с. ": "село ",
    "с-ло": "село",
    "с.-ло": "село",
    "с. -ло": "село",
    "сел": "село",
    "с.л": "село",
    "с-л": "село",
    "с.л.": "село",
    "с.-л.": "село",
    "с. -л.": "село",
    "село ": "село ",
    "селом": "село",
    "сель": "село",
    "с.е.": "село",
    "с.о": "село",
}

address = "446541, Самарская область, Сергиевский район, с. Сергиевск, ул. Ленина, д. 75 А, строение 4, павильон, 1 этаж, к. № 1-6"
normalized_address = normalize_address(address, region_dict, city_dict, street_dict, building_dict, district_dict, village_dict)
print(normalized_address)


446541, Самарская обл Сергиевский р-н, село. Сергиевск, ул Ленина, д. 75 А, стр 4, павильон, 1 этаж, к. № 1-6


In [46]:
# Применение функции clean_address к столбцу адресов
df['address_clear'] = df['address'].apply(clean_address)

# Применение функции normalize_address к столбцу address_clear и помещение результата в столбец address_clear_2
df['address_clear_2'] = df['address_clear'].apply(normalize_address, region_dict=region_dict, city_dict=city_dict, street_dict=street_dict, building_dict=building_dict, district_dict=district_dict, village_dict=village_dict)

# Удаление промежуточного столбца address_clear, если он больше не нужен
df.drop(columns=['address_clear'], inplace=True)

# Вставка столбца address_clear_2 после столбца address
df.insert(df.columns.get_loc('address') + 1, 'address_clear_2', df.pop('address_clear_2'))



In [35]:
pd.set_option('display.max_colwidth', None)

In [47]:
df[['address', 'address_clear_2']].head(20)

,address,address_clear_2
0,"Чувашская Республика - чувашия, Мариинский Посад г, Николаева ул, дом 54","Чувашская Республика - чувашия, Мариинский Посад г, Николаева ул, д 54"
1,"678995, Саха /Якутия/ Респ, Нерюнгри г, Серебряный Бор п, дом № 304","Саха /Якутия/ Респ, Нерюнги г, Серебряный Бор п, дом № 304"
2,"618909, Пермский край, Лысьвенский городской округ, г. Лысьва, ул. Чапаева, д. 21, литер А, помещения №№ 1-5 (по поэтажному плану) на первом этаже","Пермский край, Лысьвенский г одской округ, г Лысьва, ул Чапаева, д 21, литер А"
3,"Екб, Верхняя Пышма, Успенский пр-кт., 20","Екб, Верхняя Пышма, Успенский пр-кт., 20"
4,"654054, Кемеровская область - Кузбасс, г. Новокузнецк, Новоильинский район, пр-кт Запсибовцев, зд. 6-Б, помещ.2","Кемеровская обл - Кузбасс, г Новокузнецк, пр-кт Запсибовцев, зд 6-Б"
5,"654044, Кемеровская область - Кузбасс, Новокузнецкий городской округ, г. Новокузнецк, Новоильинский р-н, ул. Новоселов, зд. 29, помещ. 4А (помещения 13, 14)","Кемеровская обл - Кузбасс, Новокузнецкий г одской округ, г Новокузнецк, ул Новоселов, зд 29"
6,"446541, Самарская область, Сергиевский район, с. Сергиевск, ул. Ленина, д. 75 А, строение 4, павильон, 1 этаж, к. № 1-6","Самарская обл село. Сергиевск, ул Ленина, д 75 А, стр. 4, к. № 1-6"
7,"г.Петрозаводск, Лесной проспект( ТЦ Лотос-Плаза 3-я очередь)","г Петрозаводск, Лесной проспект"
8,"629830, Ямало-Ненецкий автономный округ, г. Губкинский, мкр-н 12, д. 36","Ямало-Ненецкий автономный округ, г г убкинский, д 36"
9,"000000, Брянская область, Карачевский район, г. Карачев, ул. Первомайская, дом 92/27","Брянская обл г Карачев, ул Первомайская, д 92/27"


In [ ]:
# Сохраняем датафрейм в файл Excel
df.to_excel('/mnt/d/Doc_to_day/20240419/EntitiesEndpoints_fresh_cleaned_step_2.xlsx', index=False)


In [1]:
pip install tiktoken

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 4.2 MB/s eta 0:00:00a 0:00:010m
Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
import tiktoken

def count_tokens(text):
    # Инициализация токенизатора для нужной модели
    encoding = tiktoken.encoding_for_model("gpt-4")
    
    # Токенизация текста
    tokens = encoding.encode(text)
    
    # Подсчет количества токенов
    num_tokens = len(tokens)
    
    return num_tokens

def process_excel(file_path, sheet_name, column_name, filter_column1, filter_value1, filter_column2, filter_value2):
    # Загрузка Excel файла
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    
    # Фильтрация строк на основе условий
    filtered_df = df[(df[filter_column1] == filter_value1) & (df[filter_column2] == filter_value2)]
    
    # Подсчет количества токенов в отфильтрованных строках
    total_tokens = 0
    for index, row in filtered_df.iterrows():
        text = str(row[column_name])
        total_tokens += count_tokens(text)
    
    # Подсчет количества строк, которые удовлетворяют условиям
    row_count = len(filtered_df)
    
    return row_count, total_tokens

# Пример использования
file_path = '/mnt/d/Doc_To_Date/20240516/Остати точек Опелла.xlsx'  # Замените на путь к вашему файлу
sheet_name = 'Sheet1'  # Замените на имя вашего листа
column_name = 'address_clear_2'  # Название колонки, из которой брать текст для подсчета токенов
filter_column1 = 'broken_address'
filter_value1 = 'ЛОЖЬ'
filter_column2 = 'address_dadata'
filter_value2 = ' Address not found'

row_count, total_tokens = process_excel(file_path, sheet_name, column_name, filter_column1, filter_value1, filter_column2, filter_value2)
print(f"Количество строк: {row_count}")
print(f"Общее количество токенов: {total_tokens}")

Количество строк: 0
Общее количество токенов: 0


In [8]:
import pandas as pd

# Функция для загрузки и фильтрации данных из Excel
def process_excel(file_path, sheet_name, column_name, filter_column1, filter_value1, filter_column2, filter_value2):
    # Загрузка Excel файла
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    
    # Фильтрация строк на основе условий
    filtered_df = df[(df[filter_column1] == filter_value1) & (df[filter_column2] == filter_value2)]
    
    # Подсчет количества строк, которые удовлетворяют условиям
    row_count = len(filtered_df)
    
    return row_count, filtered_df

# Пример использования
file_path = '/mnt/d/Doc_To_Date/20240516/Остати точек Опелла.xlsx'  # Замените на путь к вашему файлу
sheet_name = 'Sheet1'  # Замените на имя вашего листа
column_name = 'address_clear_2'  # Название колонки, из которой брать текст для подсчета токенов
filter_column1 = 'broken_address'
filter_value1 = 'ЛОЖЬ'
filter_column2 = 'address_dadata'
filter_value2 = 'Address not found'

row_count, filtered_df = process_excel(file_path, sheet_name, column_name, filter_column1, filter_value1, filter_column2, filter_value2)
print(f"Количество строк: {row_count}")
print(filtered_df.head())

Количество строк: 0
Empty DataFrame
Columns: [Unnamed: 0, id, inn, address, address_clear_2, broken_address, new_fias, id_from, address_dadata, entityId, type, addressId, regionCode, markingBranchId, markingStatus, warehouse, fiasId, latitude, longitude, brick1, brick2, brick3, oneKeyId, status, updateIncrement, data, source, createdAt, updatedAt, deletedAt, clientId, title, whExternalId]
Index: []

[0 rows x 33 columns]


In [11]:
import pandas as pd

# Загрузка Excel файла и отображение первых строк для проверки
file_path = '/mnt/d/Doc_To_Date/20240516/Остати точек Опелла.xlsx'  # Замените на путь к вашему файлу
sheet_name = 'Sheet1'  # Замените на имя вашего листа

df = pd.read_excel(file_path, sheet_name=sheet_name)
print("Первые 5 строк файла:")
print(df.head())

# Проверим уникальные значения в столбцах фильтрации без лишних пробелов
print("\nУникальные значения в 'broken_address':")
print(df['broken_address'].unique())

print("\nУникальные значения в 'address_dadata' (с удалением лишних пробелов):")
df['address_dadata'] = df['address_dadata'].str.strip()
print(df['address_dadata'].unique())

# Теперь попробуем отфильтровать данные по условиям и отобразим первые строки отфильтрованных данных
filter_column1 = 'broken_address'
filter_value1 = False
filter_column2 = 'address_dadata'
filter_value2 = 'Address not found'

filtered_df = df[(df[filter_column1] == filter_value1) & (df[filter_column2] == filter_value2)]

print(f"\nКоличество строк после фильтрации: {len(filtered_df)}")
print(filtered_df.head())



Первые 5 строк файла:
   Unnamed: 0       id         inn  \
0           0  1014799  1434038369   
1           1  1011537  5906086550   
2           2  1014800  5902199785   
3           3  1011538  4217146394   
4           4  1011539  4217178886   

                                             address  \
0  678995, Саха /Якутия/ Респ, Нерюнгри г, Серебр...   
1  618909, Пермский край, Лысьвенский городской о...   
2           Екб, Верхняя Пышма, Успенский пр-кт., 20   
3  654054, Кемеровская область - Кузбасс, г. Ново...   
4  654044, Кемеровская область - Кузбасс, Новокуз...   

                                     address_clear_2  broken_address new_fias  \
0  Саха /Якутия/ Респ, Нерюнги г, Серебряный Бор ...           False      NaN   
1  Пермский край, Лысьвенский г одской округ, г Л...           False      NaN   
2           Екб, Верхняя Пышма, Успенский пр-кт., 20           False      NaN   
3  Кемеровская обл - Кузбасс, г Новокузнецк, пр-к...           False      NaN   
4  Кеме

In [12]:
import pandas as pd
import tiktoken

def count_tokens(text):
    # Инициализация токенизатора для нужной модели
    encoding = tiktoken.encoding_for_model("gpt-4")
    
    # Токенизация текста
    tokens = encoding.encode(text)
    
    # Подсчет количества токенов
    num_tokens = len(tokens)
    
    return num_tokens

# Загрузка Excel файла и отображение первых строк для проверки
file_path = '/mnt/d/Doc_To_Date/20240516/Остати точек Опелла.xlsx'  # Замените на путь к вашему файлу
sheet_name = 'Sheet1'  # Замените на имя вашего листа

df = pd.read_excel(file_path, sheet_name=sheet_name)
df['address_dadata'] = df['address_dadata'].str.strip()

# Условия фильтрации
filter_column1 = 'broken_address'
filter_value1 = False
filter_column2 = 'address_dadata'
filter_value2 = 'Address not found'

filtered_df = df[(df[filter_column1] == filter_value1) & (df[filter_column2] == filter_value2)]
print(f"\nКоличество строк после фильтрации: {len(filtered_df)}")

# Подсчет количества токенов в отфильтрованных строках
total_tokens = 0
for index, row in filtered_df.iterrows():
    text = str(row['address_clear_2'])
    total_tokens += count_tokens(text)

print(f"Общее количество токенов: {total_tokens}")



Количество строк после фильтрации: 27317
Общее количество токенов: 829169
